In [ ]:
import os
import librosa
import librosa.display
import matplotlib.pyplot as plt
import numpy as np

real_path = r"\for-2seconds\training\real"
fake_path = r"\for-2seconds\training\fake"
real = []
fake = []

vreal_path = r"\for-2seconds\validation\real"
vfake_path = r"\for-2seconds\validation\fake"
vreal = []
vfake = []

treal_path = r"\for-2seconds\testing\real"
tfake_path = r"\for-2seconds\testing\fake"
treal = []
tfake = []


In [ ]:
def get_spectrogram(folder_path):
    specs = []    
    for filename in os.listdir(folder_path):
        audio, sr = librosa.load(os.path.join(folder_path, filename), sr=None)
        audio_parts = np.array_split(audio, 4)
        for part in audio_parts:
            spectrogram = abs(librosa.stft(y=part))
            spectrogram = librosa.amplitude_to_db(spectrogram, ref=np.max)
            specs.append(spectrogram)
    return specs

In [ ]:
realS = get_spectrogram(real_path)
fakeS = get_spectrogram(fake_path)

vrealS = get_spectrogram(vreal_path)
vfakeS = get_spectrogram(vfake_path)

trealS = get_spectrogram(treal_path)
tfakeS = get_spectrogram(tfake_path)

In [ ]:
import numpy as np

def split_of_4(array, ratio=0.25):
    N = len(array)
    split_size = int(ratio * N) 
    split_size = split_size - (split_size % 4)  

    part1 = array[split_size:]  
    part2 = array[:split_size]  

    return part1, part2

X_trainR, X_ensembleR = split_of_4(realS)
X_trainF, X_ensembleF = split_of_4(fakeS)


print("Розмір першої частини:", len(X_trainR))  
print("Розмір другої частини:", len(X_ensembleR))  


In [ ]:
from sklearn.model_selection import train_test_split

real_labels = np.zeros(len(X_trainR))
fake_labels = np.ones(len(X_trainF))

vreal_labels = np.zeros(len(vrealS))
vfake_labels = np.ones(len(vfakeS))

X_train = np.concatenate((X_trainR, X_trainF), axis=0)
y_train = np.concatenate((real_labels, fake_labels), axis=0)

X_val = np.concatenate((vrealS, vfakeS), axis=0)
y_val = np.concatenate((vreal_labels, vfake_labels), axis=0)

shuffle_index_train = np.random.permutation(len(X_train))
X_train_shuffled = X_train[shuffle_index_train]
y_train_shuffled = y_train[shuffle_index_train]

shuffle_index_val = np.random.permutation(len(X_val))
X_val_shuffled = X_val[shuffle_index_val]
y_val_shuffled = y_val[shuffle_index_val]

X_train = X_train_shuffled[..., np.newaxis]
X_test = X_val_shuffled[..., np.newaxis]

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Dropout

model = Sequential([
    Conv2D(16, (3, 3), activation='relu', input_shape=(X_train.shape[1:])),
    MaxPooling2D((2, 2)),
    Conv2D(32, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

history = model.fit(X_train, y_train_shuffled, epochs=10, batch_size=32, validation_data=(X_test, y_val_shuffled))


In [ ]:
X_ensembleRSpred = model.predict(np.array(X_ensembleR))
X_ensembleFSpred = model.predict(np.array(X_ensembleF))

vrealSpred = model.predict(np.array(vrealS))
vfakeSpred = model.predict(np.array(vfakeS))

trealSpred = model.predict(np.array(trealS))
tfakeSpred = model.predict(np.array(tfakeS))

In [ ]:
import pandas as pd
pd.DataFrame(X_ensembleRSpred).to_csv('X_ensembleRSpred.csv', index=False)
pd.DataFrame(X_ensembleFSpred).to_csv('X_ensembleFSpred.csv', index=False)
pd.DataFrame(vrealSpred).to_csv('vrealSpred.csv', index=False)
pd.DataFrame(vfakeSpred).to_csv('vfakeSpred.csv', index=False)
pd.DataFrame(trealSpred).to_csv('trealSpred.csv', index=False)
pd.DataFrame(tfakeSpred).to_csv('tfakeSpred.csv', index=False)